In [ ]:
import json
import re
import subprocess

from multiprocessing import Process, JoinableQueue as Queue
from os import listdir, path
from queue import Empty as QueueEmptyException

CAMERA_DB_PATH = "./metadata/cameras.json"

VIDEO_PATH = "/home/vmubuntu/OitoDeJaneiro/vids/0801-500"
DB_PATH = "./metadata/videos.json"

In [ ]:
with open(DB_PATH, "r") as f:
  video_data = json.load(f)

with open(CAMERA_DB_PATH, "r") as f:
  camera_data = json.load(f)

In [ ]:
def localtime_to_epoch(s):
  return 0

In [ ]:
to_process = Queue()

for io_dir in camera_data.keys():
  input_dir_path = path.join(VIDEO_PATH, io_dir)
  input_files = sorted([f for f in listdir(input_dir_path) if f.endswith("mp4")])

  for io_file in input_files:
    input_file_path = path.join(input_dir_path, io_file)

    if io_file not in video_data:
      video_data[io_file] = {
        "name": io_file,
        "dir": io_dir,
      }

    file_data = video_data[io_file]
    vid = None
    if not ("length_seconds" in file_data and "length_frames" in file_data):
      if vid is None:
        vid = cv2.VideoCapture(input_file_path)

      fps = vid.get(cv2.CAP_PROP_FPS)
      length_frames = vid.get(cv2.CAP_PROP_FRAME_COUNT)
      file_data["length_frames"] = length_frames

      vid.set(cv2.CAP_PROP_POS_AVI_RATIO, 1)
      length_seconds_seek = vid.get(cv2.CAP_PROP_POS_MSEC) // 1000
      length_seconds_fps = length_frames // fps

      if abs(length_seconds_fps - length_seconds_seek) > 2:
        print("ERROR: video length", io_file, length_seconds_fps, length_seconds_seek)
      else:
        file_data["length_seconds"] = length_seconds_fps

    if not ("time_start" in file_data and "time_end" in file_data):
      if vid is None:
        vid = cv2.VideoCapture(input_file_path)
        pass
      # get 3x15 frames
      # crop frames
      # process frame groups, get localtime strings and dates
      # check consistency
      # get epoch times
      # check continuous

In [ ]:
with open(DB_PATH, "w") as f:
  json.dump(video_data, f, indent=2)